<a href="https://colab.research.google.com/github/imjoung/DataScience/blob/main/ml/12_DBSCAN_%EA%B3%A0%EB%93%B1%ED%95%99%EA%B5%90_%EC%A7%84%ED%95%99%EB%A5%A0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%A4%91%ED%95%99%EA%B5%90_%EA%B5%B0%EC%A7%91%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

문제 정의 : 고등학교 진학률 데이터를 활용한 중학교 DBSCAN 군집분석 및 지도 시각화

In [1]:
# 기본 라이브러리 불러오기
import pandas as pd
import folium

### step 1) 데이터 준비하기

In [2]:
# 서울시내 중학교 진학률 데이터셋
df = pd.read_excel('/content/2016_middle_shcool_graduates_report.xlsx',engine='openpyxl',header=0)


### step 2) 데이터 탐색하기

In [3]:
# 열 이름 배열을 출력
df.columns.values

array(['Unnamed: 0', '지역', '학교명', '코드', '유형', '주야', '남학생수', '여학생수', '일반고',
       '특성화고', '과학고', '외고_국제고', '예고_체고', '마이스터고', '자사고', '자공고', '기타진학',
       '취업', '미상', '위도', '경도'], dtype=object)

In [4]:
# 데이터 살펴보기
df.head()

,Unnamed: 0,지역,학교명,코드,유형,주야,남학생수,여학생수,일반고,특성화고,과학고,외고_국제고,예고_체고,마이스터고,자사고,자공고,기타진학,취업,미상,위도,경도
0,0,성북구,서울대학교사범대학부설중학교,3,국립,주간,277,0,0.585,0.148,0.018,0.007,0.000,0.011,0.227,0.000,0.004,0,0.000,37.594942,127.038909
1,1,종로구,서울대학교사범대학부설여자중학교,3,국립,주간,0,256,0.680,0.199,0.000,0.035,0.008,0.000,0.043,0.004,0.031,0,0.000,37.577473,127.003857
2,2,강남구,개원중학교,3,공립,주간,170,152,0.817,0.047,0.009,0.012,0.003,0.006,0.090,0.003,0.009,0,0.003,37.491637,127.071744
3,3,강남구,개포중학교,3,공립,주간,83,72,0.755,0.097,0.013,0.013,0.019,0.019,0.065,0.000,0.019,0,0.000,37.480439,127.062201
4,4,서초구,경원중학교,3,공립,주간,199,212,0.669,0.017,0.007,0.010,0.005,0.000,0.282,0.000,0.010,0,0.000,37.510750,127.008900


In [5]:
# 데이터 자료형 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  415 non-null    int64  
 1   지역          415 non-null    object 
 2   학교명         415 non-null    object 
 3   코드          415 non-null    int64  
 4   유형          415 non-null    object 
 5   주야          415 non-null    object 
 6   남학생수        415 non-null    int64  
 7   여학생수        415 non-null    int64  
 8   일반고         415 non-null    float64
 9   특성화고        415 non-null    float64
 10  과학고         415 non-null    float64
 11  외고_국제고      415 non-null    float64
 12  예고_체고       415 non-null    float64
 13  마이스터고       415 non-null    float64
 14  자사고         415 non-null    float64
 15  자공고         415 non-null    float64
 16  기타진학        415 non-null    float64
 17  취업          415 non-null    int64  
 18  미상          415 non-null    float64
 19  위도          415 non-null    f

중학교 위치 지도 시각화

In [7]:
mschool_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

# 중학교 위치정보를 CircleMarker로 표시
for name, lat, lng in zip(df.학교명, df.위도, df.경도):
    folium.CircleMarker([lat, lng],
                        radius=5,              # 원의 반지름
                        color='brown',         # 원의 둘레 색상
                        fill=True,
                        fill_color='coral',    # 원을 채우는 색
                        fill_opacity=0.7,      # 투명도    
                        popup=name
    ).add_to(mschool_map)
mschool_map

### step 3) 데이터 전처리

In [8]:
# 원핫인코딩(더미 변수)
from sklearn import preprocessing    

label_encoder = preprocessing.LabelEncoder()     # label encoder 생성

onehot_location = label_encoder.fit_transform(df['지역'])
onehot_code = label_encoder.fit_transform(df['코드'])
onehot_type = label_encoder.fit_transform(df['유형'])
onehot_day = label_encoder.fit_transform(df['주야'])

df['location'] = onehot_location
df['code'] = onehot_code
df['type'] = onehot_type
df['day'] = onehot_day

df.head()


,Unnamed: 0,지역,학교명,코드,유형,주야,남학생수,여학생수,일반고,특성화고,과학고,외고_국제고,예고_체고,마이스터고,자사고,자공고,기타진학,취업,미상,위도,경도,location,code,type,day
0,0,성북구,서울대학교사범대학부설중학교,3,국립,주간,277,0,0.585,0.148,0.018,0.007,0.000,0.011,0.227,0.000,0.004,0,0.000,37.594942,127.038909,16,0,1,0
1,1,종로구,서울대학교사범대학부설여자중학교,3,국립,주간,0,256,0.680,0.199,0.000,0.035,0.008,0.000,0.043,0.004,0.031,0,0.000,37.577473,127.003857,22,0,1,0
2,2,강남구,개원중학교,3,공립,주간,170,152,0.817,0.047,0.009,0.012,0.003,0.006,0.090,0.003,0.009,0,0.003,37.491637,127.071744,0,0,0,0
3,3,강남구,개포중학교,3,공립,주간,83,72,0.755,0.097,0.013,0.013,0.019,0.019,0.065,0.000,0.019,0,0.000,37.480439,127.062201,0,0,0,0
4,4,서초구,경원중학교,3,공립,주간,199,212,0.669,0.017,0.007,0.010,0.005,0.000,0.282,0.000,0.010,0,0.000,37.510750,127.008900,14,0,0,0


### step 4) DBSCAN 군집 모형 - sklearn 사용

In [10]:
# sklearn 라이브러리에서 cluster 군집 모형 가져오기 
from sklearn import cluster

# 분석에 사용할 속성을 선택 (과학고, 외고국제고, 자사고 진학률)
columns_list = [9, 10, 13]
X = df.iloc[:, columns_list]
print(X[:5])

    특성화고    과학고  마이스터고
0  0.148  0.018  0.011
1  0.199  0.000  0.000
2  0.047  0.009  0.006
3  0.097  0.013  0.019
4  0.017  0.007  0.000


In [12]:
# 설명 변수 데이터를 정규화
X = preprocessing.StandardScaler().fit(X).transform(X)
X

array([[-0.01637623,  2.02375287,  0.76167223],
       [ 0.47947907, -0.65047921, -0.69559254],
       [-0.99836417,  0.68663683,  0.09927916],
       ...,
       [-1.45532886, -0.65047921, -0.69559254],
       [-1.45532886, -0.65047921, -0.69559254],
       [-1.27059845, -0.65047921,  0.62919362]])

In [14]:
# DBSCAN 모형 객체 생성
dbm = cluster.DBSCAN(eps=0.2, min_samples=5)
# 모형 학습
dbm.fit(X)

DBSCAN(algorithm='auto', eps=0.2, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=5, n_jobs=None, p=None)

In [15]:
# 예측(군집)
cluster_label = dbm.labels_
print(pd.unique(cluster_label)) # 13개로 클러스터링

[-1  0  1  5  2  3  4  6  8  7  9 10 11]


In [16]:
# 예측결과를 데이터프레임에 추가
df['Cluster'] = cluster_label
print(df.head())

   Unnamed: 0   지역               학교명  코드  ... code type  day  Cluster
0           0  성북구    서울대학교사범대학부설중학교   3  ...    0    1    0       -1
1           1  종로구  서울대학교사범대학부설여자중학교   3  ...    0    1    0        0
2           2  강남구             개원중학교   3  ...    0    0    0       -1
3           3  강남구             개포중학교   3  ...    0    0    0       -1
4           4  서초구             경원중학교   3  ...    0    0    0        1

[5 rows x 26 columns]


In [17]:
# 클러스터 값으로 그룹화하고, 그룹별로 내용 출력 (첫 5행만 출력)
grouped_cols = [0, 1, 3] + columns_list
grouped = df.groupby('Cluster')
for key, group in grouped:
    print('* key :', key)
    print('* number :', len(group))    
    print(group.iloc[:, grouped_cols].head())
    print('\n')


* key : -1
* number : 234
   Unnamed: 0   지역  코드   특성화고    과학고  마이스터고
0           0  성북구   3  0.148  0.018  0.011
2           2  강남구   3  0.047  0.009  0.006
3           3  강남구   3  0.097  0.013  0.019
6           6  강남구   3  0.015  0.015  0.000
7           7  강남구   3  0.000  0.032  0.000


* key : 0
* number : 70
    Unnamed: 0   지역  코드   특성화고  과학고  마이스터고
1            1  종로구   3  0.199  0.0    0.0
13          13  서초구   3  0.032  0.0    0.0
23          23  강남구   3  0.025  0.0    0.0
28          28  서초구   3  0.040  0.0    0.0
29          29  강남구   3  0.051  0.0    0.0


* key : 1
* number : 7
    Unnamed: 0   지역  코드   특성화고    과학고  마이스터고
4            4  서초구   3  0.017  0.007    0.0
24          24  강남구   3  0.026  0.007    0.0
27          27  강남구   3  0.006  0.006    0.0
53          53  강동구   3  0.020  0.007    0.0
68          68  송파구   3  0.011  0.007    0.0


* key : 2
* number : 6
    Unnamed: 0   지역  코드   특성화고    과학고  마이스터고
8            8  강남구   3  0.018  0.013    0.0
12          12  

In [19]:
# 그래프로 표현 - 시각화
colors = {-1:'gray', 0:'coral', 1:'blue', 2:'green', 3:'red', 4:'purple', 
          5:'orange', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan', 11:'tan'}

cluster_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster_map)
cluster_map

In [20]:
# X2 데이터셋에 대하여 위의 과정을 반복(과학고, 외고국제고, 자사고 진학률 + 유형)
columns_list2 = [9, 10, 13, 22]
X2 = df.iloc[:, columns_list2]
print(X2[:5])

    특성화고    과학고  마이스터고  code
0  0.148  0.018  0.011     0
1  0.199  0.000  0.000     0
2  0.047  0.009  0.006     0
3  0.097  0.013  0.019     0
4  0.017  0.007  0.000     0


In [21]:
X2 = preprocessing.StandardScaler().fit(X2).transform(X2)
dbm2 = cluster.DBSCAN(eps=0.2, min_samples=5)
dbm2.fit(X2)  
df['Cluster2'] = dbm2.labels_   

grouped2_cols = [0, 1, 3] + columns_list2
grouped2 = df.groupby('Cluster2')
for key, group in grouped2:
    print('* key :', key)
    print('* number :', len(group))    
    print(group.iloc[:, grouped2_cols].head())
    print('\n')


* key : -1
* number : 238
   Unnamed: 0   지역  코드   특성화고    과학고  마이스터고  code
0           0  성북구   3  0.148  0.018  0.011     0
2           2  강남구   3  0.047  0.009  0.006     0
3           3  강남구   3  0.097  0.013  0.019     0
6           6  강남구   3  0.015  0.015  0.000     0
7           7  강남구   3  0.000  0.032  0.000     0


* key : 0
* number : 70
    Unnamed: 0   지역  코드   특성화고  과학고  마이스터고  code
1            1  종로구   3  0.199  0.0    0.0     0
13          13  서초구   3  0.032  0.0    0.0     0
23          23  강남구   3  0.025  0.0    0.0     0
28          28  서초구   3  0.040  0.0    0.0     0
29          29  강남구   3  0.051  0.0    0.0     0


* key : 1
* number : 7
    Unnamed: 0   지역  코드   특성화고    과학고  마이스터고  code
4            4  서초구   3  0.017  0.007    0.0     0
24          24  강남구   3  0.026  0.007    0.0     0
27          27  강남구   3  0.006  0.006    0.0     0
53          53  강동구   3  0.020  0.007    0.0     0
68          68  송파구   3  0.011  0.007    0.0     0


* key : 2
* number : 

In [23]:
cluster2_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster2):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster2_map)

# 지도를 html 파일로 저장하기
cluster2_map.save('seoul_mschool_cluster2.html')

In [24]:
# X3 데이터셋에 대하여 위의 과정을 반복(과학고, 외고_국제고)
columns_list3 = [9, 10]
X3 = df.iloc[:, columns_list3]
print(X3[:5])
print('\n')

X3 = preprocessing.StandardScaler().fit(X3).transform(X3)
dbm3 = cluster.DBSCAN(eps=0.2, min_samples=5)
dbm3.fit(X3)  
df['Cluster3'] = dbm3.labels_   

grouped3_cols = [0, 1, 3] + columns_list3
grouped3 = df.groupby('Cluster3')
for key, group in grouped3:
    print('* key :', key)
    print('* number :', len(group))    
    print(group.iloc[:, grouped3_cols].head())
    print('\n')
cluster3_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster3):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster3_map)

# 지도를 html 파일로 저장하기
cluster3_map.save('seoul_mschool_cluster3.html')
cluster3_map


    특성화고    과학고
0  0.148  0.018
1  0.199  0.000
2  0.047  0.009
3  0.097  0.013
4  0.017  0.007


* key : -1
* number : 73
    Unnamed: 0   지역  코드   특성화고    과학고
0            0  성북구   3  0.148  0.018
3            3  강남구   3  0.097  0.013
6            6  강남구   3  0.015  0.015
7            7  강남구   3  0.000  0.032
10          10  강남구   3  0.000  0.031


* key : 0
* number : 205
    Unnamed: 0   지역  코드   특성화고  과학고
1            1  종로구   3  0.199  0.0
13          13  서초구   3  0.032  0.0
22          22  강남구   3  0.158  0.0
23          23  강남구   3  0.025  0.0
28          28  서초구   3  0.040  0.0


* key : 1
* number : 104
    Unnamed: 0   지역  코드   특성화고    과학고
2            2  강남구   3  0.047  0.009
5            5  강남구   3  0.066  0.007
11          11  강남구   3  0.064  0.007
15          15  서초구   3  0.046  0.009
32          32  서초구   3  0.131  0.003


* key : 2
* number : 11
    Unnamed: 0   지역  코드   특성화고    과학고
4            4  서초구   3  0.017  0.007
9            9  강남구   3  0.000  0.006
24         